In [1]:
import tensorflow as tf
import tflearn
import numpy as np
import gensim

In [2]:
class LSTM:
    def __init__(self, input_sentence, vocabulary_size=200, num_nodes=128, 
                 batch_size=20, num_unrollings=2, name=""):
        '''
        the LSTM process
        '''
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        self.num_unrollings = num_unrollings
        self.batch_size = batch_size
        self.name = name
        
        with tf.variable_scope("{}_LSTM_var".format(self.name) ,reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0.1))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False)
            '''self.saved_state_backward = tf.get_variable("saved_state_backward", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False, name="backward_state")'''
                      
        
        
        self.train_data = input_sentence
        #Since now input_sentence is a list of word vectors, 
        #we do not need following 3 lines anymore.
        '''for _ in range(self.num_unrollings + 1):
            self.train_data.append(
                tf.placeholder(tf.float32, shape=[self.batch_size, self.vocabulary_size]))'''
        self.train_inputs = self.train_data[:-]
        self.train_labels = self.train_data[1:]
        
        self.outputs = list()
        self.output = self.saved_output
        self.state = self.saved_state
        
        for i in self.train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(self.output)
        
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
        self.reuse = True
        
    def lstm_cell(self, inputs, hidden_layer, state): 
        
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state   
    
    '''def lstm_process(self, inputs, hidden_layer, state):
        for i in train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(output)

        return outputs'''
        '''with tf.control_dependencies([self.saved_output.assign(output),
                                     self.saved_state.assign(state)]):
            
            logits = tf.matmul(tf.concat(0, outputs), self.w) + self.b
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(tf.concat(0, logits), 
                                                        tf.concat(0, train_labels), name="loss"))
            
            global_step = tf.Variable(0)
            learning_rate = tf.train.exponential_decay(
                10.0, global_step, 5000, 0.1, staircase=True)
            optimizer = tf.train.RMSPropOptimizer(learning_rate)
            gradient, v = zip(*optimizer.compute_gradients(loss))
            
            
            tf.add_to_collection("loss", loss)
            tf.add_to_collection("logits", logits)
            tf.add_to_collection("train_labels", train_labels)
            
        '''
        #It seems we do not need those lines above
        
    def train(self):
        pass
    
    def __call__(self, inputs):
        self.lstm_process(inputs)
        

In [7]:
class gen_LSTM():
    def __init__(self, bi_hidden, max_len=32, name=""):
        self.bi_hidden = bi_hidden
        self.name = name
        self.reuse = False
        
        with tf.variable_scope("{}_gen_LSTM".format(self.name),
                              reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0.1))
                self.ic = tf.get_variable("ic", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.fc = tf.get_variable("fc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.oc = tf.get_variable("oc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                self.cc = tf.get_variable("cc", [num_nodes*2, num_nodes],
                                         tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, WORD_SIZE], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [WORD_SIZE], tf.float32,
                                        tf.constant_initializer(0.1))
                
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False)
            
        #now start the LSTM process
            c = attention(self.bi_hidden, self.saved_state)
            
        self.reuse = True
        
    def gen_lstm_cell(self, inputs, hidden_layer, state, c): 
        #single step LSTM with context vector
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im)\
                                    + tf.matmul(self.ic, c) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm)\
                                    + tf.matmul(self.fc, c) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm)\
                                  + tf.matmul(self.cc, c) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state
        
    def attention(self, h, s):
        with tf.variable_scope("attention_module", reuse=self.reuse):
            self.Ua = tf.get_variable("Ua", [int(h[0].get_shape()[1])/2, int(h[0].get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            self.Wa = tf.get_variable("Wa", [int(s.get_shape()[1]), int(s.get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            #the following bias does not showed in original paper
            self.Ba = tf.get_variable("Ba", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            self.Va = tf.get_variable("Va", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            
            for h_j in h:
                attn_mlp = tf.tanh(
                    tf.matmul(self.Wa, tf.transpose(s)) + tf.matmul(self.Ua, tf.transpose(h_j)))
                attn_logits.append(tf.reshape(
                        tf.matmul(self.Va, attn_mlp), shape=[1])[0])
                
            self.a = tf.reshape(tf.nn.softmax(self.attn_logits),
                                shape=[len(h)], name="attention_value")
            

In [3]:
class attention:
    def __init__(self, h, s, name=""):
        '''
        Caculate the attention weight.
        TODO:
            I think it seems appropriate to transform the class 
            into a function...
        arguments:
            h: all the encoder's hidden states, 
            each of them has a size of [2*Hidden_size, Batch_size]
            s: [2*Hidden_size, Batch_size], decoder's hidden state
        '''
        self.name = name
        self.attn_logits = list()
        self.a = list()
        self.reuse = False
        
        #some variables
        with tf.variable_scope("{}_attention".format(self.name),
                              reuse=self.reuse):
            self.Ua = tf.get_variable("Ua", [int(h[0].get_shape()[1])/2, int(h[0].get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            self.Wa = tf.get_variable("Wa", [int(s.get_shape()[1]), int(s.get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            #the following bias does not showed in original paper
            self.Ba = tf.get_variable("Ba", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            self.Va = tf.get_variable("Va", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            
            #caculate the attention weight
            for h_j in h:
                self.attn_mlp = tf.tanh(
                    tf.matmul(self.Wa, tf.transpose(s)) + tf.matmul(self.Ua, tf.transpose(h_j)))
                self.attn_logits.append(tf.reshape(
                        tf.matmul(self.Va, self.attn_mlp), shape=[1])[0])
                
            self.a = tf.reshape(tf.nn.softmax(self.attn_logits),
                                shape=[len(h)], name="attention_value")
        self.reuse = True
    def get_attention():
        return self.a

In [4]:
class Discriminator():
    def __init__(self, s, conv_sizes, num_filters, dropout_keep_prob,
                 reuse=False, name=""):
        '''
        The Discriminator, uses Convolutional Neural Network to 
        classify the input sentence. See 
        http://www.people.fas.harvard.edu/~yoonkim/data/emnlp_2014.pdf
        for detail. Those code below with some inspiration from
        [https://github.com/dennybritz/cnn-text-classification-tf/blob/master/text_cnn.py]
        
        Arguments:
        s: the sentence representation caculated by LSTM process, 
        which has a size of [batch_size, hidden_layer_size]
        '''
        self.sequence_length = int(s.get_shape()[1])
        self.reuse = reuse
        self.name = name
        self.dropout_keep_prob = dropout_keep_prob
        self.num_filters = num_filters
        self.conv_sizes = conv_sizes
        self.hidden_size = int(s.get_shape()[2])
        #print(self.hidden_size)
        self.pooled_outputs = list()
        for i, conv_size in enumerate(conv_sizes):
            with tf.variable_scope("D_conv_maxpooling_{}"
                                  .format(conv_size), reuse=self.reuse):
                #filter_shape = [conv_size, self.hidden_size, 1, num_filters]
                W = tf.get_variable("W", [conv_size, self.hidden_size, 1, num_filters],
                                   tf.float32,
                                   tf.truncated_normal_initializer(stddev=0.1))
                print("the size of {} => {}"
                      .format(W.name, W.get_shape()))
                b = tf.get_variable("b", [num_filters], tf.float32,
                                   tf.constant_initializer(0.1))
                
                conv = tf.nn.conv2d(s, W, [1,1,1,1], 'VALID', name="conv")
                print("the size of {} => {}"
                      .format(conv.name, conv.get_shape()))
                h = tf.nn.relu(conv + b)
                
                pooled = tf.nn.max_pool(h,
                                       ksize=[1, self.sequence_length-conv_size+1, 1, 1],
                                       strides=[1, 1, 1, 1],
                                       padding='VALID',
                                       name="max_pooling")
                print("the size of pooling layer {}"
                      .format(pooled.get_shape()))
            self.pooled_outputs.append(pooled)
            
        self.total_filter_num = num_filters * len(conv_sizes)
        self.h_pool = tf.concat(3, self.pooled_outputs,
                                name="{}_concat_pooling_layer".format(self.name))
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, self.total_filter_num])
        print("h_pool size => {}".format(self.h_pool_flat.get_shape()))
     
        #after convolutional layer, we can define the fully connectedmlayer
        
        with tf.variable_scope("D_fully_connected_layer", reuse=self.reuse):
            w_shape = [self.h_pool_flat.get_shape()[1], 1]
            W = tf.get_variable("W", w_shape, tf.float32,
                               tf.truncated_normal_initializer(stddev=0.1))
            b = tf.get_variable("b", [1], tf.float32,
                               tf.constant_initializer(0.1))
            logits = tf.nn.relu(tf.matmul(self.h_pool_flat, W) + b)
            print("logits size => {}".format(logits.get_shape()))
            
        self.reuse = True

In [ ]:
class Generator:
    def __init__(self, input_sentence, max_len=32, vocabulary_size=200, 
                 num_nodes=128, batch_size=16, name=""):
        self.input_sentence = input_sentence
        self.reverse_sentence = self.reverse_sentence(self.input_sentence)
        self.forward_LSTM_outputs = LSTM(self.input_sentence, 
                                        name="forward")
        self.backward_LSTM_outputs = LSTM(self.reverse_sentence,
                                         name="backward")     
        #concat the two outputs 
        
        #using attention
        self.gen_LSTM = LSTM()
        self.attention = attention()
        
    def reverse_sentence(self, input_sentence):
        return input_sentence.reverse()
    
    def single_step_LSTM():
        

In [ ]:
'''class LSTMGAN:
    def __init__(self):
        self.lstm = LSTM()
        self.d = Discriminator()'''

In [9]:
h = tf.Variable(tf.truncated_normal((1,128)))
state = tf.Variable(tf.truncated_normal((1,128)))
test_inputs = tf.placeholder(tf.float32, shape=[1, 200])

In [10]:
h_j = list()
for _ in range(10):
    h_j.append(tf.Variable(tf.truncated_normal((1, 256))))
s_j = tf.Variable(tf.truncated_normal((1, 128)))

In [11]:
A = attention(h_j, s_j, "1224")

In [12]:
s_d = tf.Variable(tf.truncated_normal((10, 200)))
s_d = tf.reshape(s_d, [1, 10, 200, 1])

In [13]:
D = Discriminator(s_d, [3], 64, 0.5, False,"ddd1d")

the size of D_conv_maxpooling_3/W:0 => (3, 200, 1, 64)
the size of D_conv_maxpooling_3/conv:0 => (1, 8, 1, 64)
the size of pooling layer (1, 1, 1, 64)
h_pool size => (1, 64)
logits size => (1, 1)


In [14]:
L = LSTM(batch_size=1, num_unrollings=2, name="TEST")

word_model = gensim.models.Word2Vec.load("models/word2vec.model")

test_word1 = word_model["china"]
test_word2 = word_model["uk"]
test_word3 = word_model["america"]

test_sentence = [test_word1, test_word2, test_word3]

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    feed_dict = dict()
    for i in range(3):
        feed_dict[L.train_data[i]] = np.reshape(test_sentence[i],(1,200))
    out = sess.run(L.outputs, feed_dict=feed_dict)
    print(out)

[array([[ -3.26242559e-02,   6.03667423e-02,  -2.56971180e-01,
          5.60867116e-02,   4.49584499e-02,   6.33594468e-02,
         -6.29332364e-01,   7.00063333e-02,   4.59783785e-02,
          7.34074712e-02,  -1.48096934e-01,   3.71790081e-02,
         -9.02559701e-03,   5.46270609e-01,   2.88277343e-02,
          1.59395486e-02,   2.25683570e-01,   1.35768533e-01,
         -4.23966087e-02,  -2.41617516e-01,  -7.49615878e-02,
         -2.07237810e-01,   2.72097379e-01,   4.00093585e-01,
          2.65516229e-02,  -5.00426348e-03,  -1.25763699e-01,
          1.30933657e-01,   3.59464320e-03,   1.26224816e-01,
          1.57776967e-01,   5.13974018e-02,   3.45506966e-02,
          2.67466903e-01,   6.48778304e-02,   3.05070996e-01,
          2.41123393e-01,   2.47940704e-01,  -4.11151379e-01,
         -7.13195419e-03,  -1.89463809e-01,  -1.07426196e-01,
          6.85911626e-03,  -3.61409247e-01,  -3.41369361e-02,
          5.66916689e-02,   1.10777006e-01,  -6.11392856e-02,
       

In [15]:
out[0].shape

(1, 128)

In [2]:
ll = [5,4,3,2,1]
ll[:-1]

[5, 4, 3, 2]